In [5]:
from transcribe import AudioRecorder

audio_recorder = AudioRecorder()
transcript = audio_recorder.run()


Listening... Press Ctrl+C to stop.
Stopping...
Transcription:
 Hi hello, hi hello, hi
